In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000

In [2]:
#GoogleNet Lstm
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the GoogleNet-LSTM model
def inception_module(x, filters):
    branch1x1 = layers.Conv1D(filters[0], 1, activation='relu', padding='same')(x)
    
    branch3x3 = layers.Conv1D(filters[1], 1, activation='relu', padding='same')(x)
    branch3x3 = layers.Conv1D(filters[2], 3, activation='relu', padding='same')(branch3x3)
    
    branch5x5 = layers.Conv1D(filters[3], 1, activation='relu', padding='same')(x)
    branch5x5 = layers.Conv1D(filters[4], 5, activation='relu', padding='same')(branch5x5)
    
    branch_pool = layers.MaxPooling1D(3, strides=1, padding='same')(x)
    branch_pool = layers.Conv1D(filters[5], 1, activation='relu', padding='same')(branch_pool)
    
    output = layers.concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

input_shape = (X_train.shape[1], X_train.shape[2])  # Adjust input shape as per your requirements

input_layer = layers.Input(shape=input_shape)
x = layers.Conv1D(6, kernel_size=5, activation='relu', padding='same')(input_layer)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(16, kernel_size=5, activation='relu', padding='same')(x)
x = layers.MaxPooling1D(pool_size=2)(x)

# Replace fully connected layers with Inception modules
x = inception_module(x, [6, 6, 8, 2, 4, 4])
x = inception_module(x, [16, 8, 12, 4, 8, 8])

x = layers.LSTM(84, return_sequences=True)(x)
x = layers.Flatten()(x)
x = layers.Dense(1, activation='linear')(x)

model = models.Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=32, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions
predictions = model.predict(X_test)

ypred = predictions.flatten()


Epoch 1/32
1131/1131 [==============================] - 14s 9ms/step - loss: 0.0124
Epoch 2/32
1131/1131 [==============================] - 11s 10ms/step - loss: 0.0012
Epoch 3/32
1131/1131 [==============================] - 11s 10ms/step - loss: 0.0011
Epoch 4/32
1131/1131 [==============================] - 11s 10ms/step - loss: 0.0010
Epoch 5/32
1131/1131 [==============================] - 11s 10ms/step - loss: 9.0077e-04
Epoch 6/32
1131/1131 [==============================] - 11s 10ms/step - loss: 8.6343e-04
Epoch 7/32
1131/1131 [==============================] - 11s 10ms/step - loss: 7.8595e-04
Epoch 8/32
1131/1131 [==============================] - 11s 10ms/step - loss: 7.7231e-04
Epoch 9/32
1131/1131 [==============================] - 11s 10ms/step - loss: 7.6275e-04
Epoch 10/32
1131/1131 [==============================] - 11s 10ms/step - loss: 7.6418e-04
Epoch 11/32
1131/1131 [==============================] - 11s 10ms/step - loss: 7.5715e-04
Epoch 12/32
1131/1131 [=============

In [3]:
# For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

# For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

MAE = 0.02 (%)
MAPE = 1.49 (%)
RMSE = 0.02 (%)
